<a href="https://colab.research.google.com/github/satijagunika/HR_Dashboard_using_Tableau/blob/main/Tableau_HR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random

# Initialize Faker
fake = Faker()

# Parameters
n_records = 8950
departments = {
    'HR': ['HR Manager', 'HR Assistant', 'HR Specialist'],
    'Engineering': ['Software Engineer', 'Senior Software Engineer', 'Engineering Manager'],
    'Sales': ['Sales Manager', 'Sales Representative', 'Sales Associate'],
    'Finance': ['Finance Manager', 'Financial Analyst', 'Accountant']
}
education_levels = {
    'HR Manager': 'Master\'s',
    'HR Assistant': 'Bachelor\'s',
    'HR Specialist': 'Bachelor\'s',
    'Software Engineer': 'Bachelor\'s',
    'Senior Software Engineer': 'Master\'s',
    'Engineering Manager': 'Master\'s',
    'Sales Manager': 'Bachelor\'s',
    'Sales Representative': 'Bachelor\'s',
    'Sales Associate': 'Associate\'s',
    'Finance Manager': 'Master\'s',
    'Financial Analyst': 'Bachelor\'s',
    'Accountant': 'Bachelor\'s'
}
performance_ratings = ['Excellent', 'Good', 'Satisfactory', 'Needs Improvement']
states_and_cities = {
    'California': ['Los Angeles', 'San Francisco', 'San Diego'],
    'Texas': ['Houston', 'Dallas', 'Austin'],
    'New York': ['New York City', 'Buffalo', 'Rochester'],
    'Florida': ['Miami', 'Orlando', 'Tampa']
}
gender_distribution = [0.46, 0.54]
overtime_distribution = [0.3, 0.7]
performance_distribution = [0.2, 0.3, 0.4, 0.1]

# Helper functions
def random_date(start, end, prop):
    return start + prop * (end - start)

def generate_employee_id(n):
    return [f'E{str(i).zfill(6)}' for i in range(1, n+1)]

def generate_hire_date(n):
    start_date = pd.to_datetime('2015-01-01')
    end_date = pd.to_datetime('2024-01-01')
    return [random_date(start_date, end_date, random.random()) for _ in range(n)]

def generate_termination_date(hire_dates):
    termination_dates = []
    for hire_date in hire_dates:
        if random.random() < 0.112:
            start_date = hire_date + pd.DateOffset(months=6)
            end_date = pd.to_datetime('2024-08-01')
            termination_date = random_date(start_date, end_date, random.random())
            termination_dates.append(termination_date)
        else:
            termination_dates.append(None)
    return termination_dates

def generate_salaries(department, job_title):
    salary_ranges = {
        'HR Manager': (70000, 90000),
        'HR Assistant': (40000, 60000),
        'HR Specialist': (50000, 70000),
        'Software Engineer': (80000, 110000),
        'Senior Software Engineer': (100000, 140000),
        'Engineering Manager': (120000, 160000),
        'Sales Manager': (70000, 100000),
        'Sales Representative': (50000, 70000),
        'Sales Associate': (35000, 50000),
        'Finance Manager': (90000, 120000),
        'Financial Analyst': (60000, 80000),
        'Accountant': (50000, 70000)
    }
    min_salary, max_salary = salary_ranges[job_title]
    return random.randint(min_salary, max_salary)

def adjusted_salary(base_salary, gender, education_level, age):
    gender_multiplier = 1.02 if gender == 'Male' else 1.0
    education_multiplier = 1.1 if education_level in ['Master\'s', 'PhD'] else 1.0
    age_multiplier = 1.05 if age >= 45 else 1.0
    return base_salary * gender_multiplier * education_multiplier * age_multiplier

# Generate dataset
data = {
    'Employee ID': generate_employee_id(n_records),
    'First Name': [fake.first_name() for _ in range(n_records)],
    'Last Name': [fake.last_name() for _ in range(n_records)],
    'Gender': np.random.choice(['Female', 'Male'], n_records, p=gender_distribution),
    'State': [],
    'City': [],
    'Hire Date': generate_hire_date(n_records),
    'Department': [],
    'Job Title': [],
    'Education Level': [],
    'Performance Rating': np.random.choice(performance_ratings, n_records, p=performance_distribution),
    'Overtime': np.random.choice(['Yes', 'No'], n_records, p=overtime_distribution),
    'Salary': [],
    'Birth Date': [],
    'Termination Date': [],
    'Adjusted Salary': []
}

# Populate State, City, Department, Job Title, Education Level, Salary, Birth Date, and Adjusted Salary
for i in range(n_records):
    state = random.choice(list(states_and_cities.keys()))
    city = random.choice(states_and_cities[state])
    data['State'].append(state)
    data['City'].append(city)

    department = random.choice(list(departments.keys()))
    job_title = random.choice(departments[department])
    data['Department'].append(department)
    data['Job Title'].append(job_title)
    data['Education Level'].append(education_levels[job_title])

    salary = generate_salaries(department, job_title)
    data['Salary'].append(salary)

    hire_date = data['Hire Date'][i]
    age_at_hire = random.randint(22, 60)
    birth_date = hire_date - pd.DateOffset(years=age_at_hire)
    data['Birth Date'].append(birth_date)

    data['Termination Date'].append(None) # Placeholder, we'll update this next

    age = pd.Timestamp('2024-08-01').year - birth_date.year
    adj_salary = adjusted_salary(salary, data['Gender'][i], data['Education Level'][i], age)
    data['Adjusted Salary'].append(adj_salary)

# Update Termination Dates
data['Termination Date'] = generate_termination_date(data['Hire Date'])

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('hr_dataset.csv', index=False)

print("Dataset generated and saved to hr_dataset.csv")


<ipython-input-4-5d4752f42399>:129: UserWarning: Discarding nonzero nanoseconds in conversion.
  birth_date = hire_date - pd.DateOffset(years=age_at_hire)
<ipython-input-4-5d4752f42399>:59: UserWarning: Discarding nonzero nanoseconds in conversion.
  start_date = hire_date + pd.DateOffset(months=6)


Dataset generated and saved to hr_dataset.csv
